# Parallelization Lab
In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

### Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
# your code here
response = requests.get(url)
print(response)

<Response [200]>


### Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup

In [6]:
# your code here
soup = BeautifulSoup(response.text, 'lxml')
print(soup.prettify()[0:100])
tags = soup.find_all("a", {"href": True})
l_unique_links = list(set([tag.get('href') for tag in tags]))

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <


### Step 3: Use list comprehensions with conditions to clean the link list.
There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

Absolute Links: Create a list of these and remove any that contain a percentage sign (%).

Relative Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).

Combine the list of absolute and relative links and ensure there are no duplicate

In [8]:
domain = 'http://wikipedia.org'

In [9]:
# your code here
l_absolute_links = [l for l in l_unique_links if (l.startswith("http") and "%" not in l)]
l_relative_links = [domain + l for l in l_unique_links if (l.startswith("/") and "%" not in l and not l.startswith("//"))]
l_links = list(set(l_absolute_links + l_relative_links))

### Step 4: Use the os library to create a folder called wikipedia and make that the current working directory.

In [10]:
import os

In [12]:
# your code here
try: 
    os.mkdir('./wiki')
except OSError as error: 
    print(error)
    
try: 
    os.chdir('./wiki')
except OSError as error: 
    print(error)  

### Step 5: Write a function called index_page that accepts a link and does the following.
Tries to request the content of the page referenced by that link.

Slugifies the filename using the slugify function from the python-slugify library and adds a .html file extension.

      If you don't already have the python-slugify library installed, you can pip install it as follows:
      $ pip3 install python-slugify.

      To import the slugify function, you would do the following: from slugify import slugify.

      You can then slugify a link as follows slugify(link).

Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.

If an exception occurs during the process above, just pass.

In [18]:
from slugify import slugify

In [19]:
# your code here
def index_page(link):
    try:
        page = requests.get(link, 'lxml')
        filename = slugify(link) + '.html'
        file = open(filename, "w")
        file.write(page.text)
    except:
        pass

### Step 6: Sequentially loop through the list of links, running the index_page function each time.
Remember to include %%time at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
# your code here
%%time

for link in l_links:
    index_page(link)
    
# CPU time: user 44 s, sys: 4.24 s, total: 48.2 s
# Wall time: 7min 1s

### Step 7: Perform the page indexing in parallel and note the difference in performance.
Remember to include %%time at the beginning of the cell so that it measures the time it takes for the cell to run.

In [28]:
import multiprocessing 

In [26]:
try: 
    os.mkdir('./wikipedia_parallelization')
except OSError as error: 
    print(error)
    
try: 
    os.chdir('./wikipedia_parallelization')
except OSError as error: 
    print(error)  

In [29]:
%%time

process_list = []
for link in l_links:
  p = multiprocessing.Process(target = index_page, args = (link,))
  p.start()
  process_list.append(p)

for process in process_list:
  process.join()

CPU times: total: 1 s
Wall time: 10.6 s
